<a href="https://colab.research.google.com/github/Pepe-Padilla/uso-de-IA/blob/main/contestadorMail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatizador de correos
> #### Componentes Intergalacticos Industriales SA.
Misión: automatizar las respuestas a los emails de solicitud de devolución de la empresa.

Objetivo
Aceptar la devolución:
- ✅ Motivos para ACEPTAR una solicitud de devolución:
 - Defecto de fabricación confirmado: El producto presenta fallos internos o de funcionamiento no atribuibles al transporte ni a un mal uso.
 - Error en el suministro: Se han entregado componentes incorrectos en cuanto a modelo, cantidad o especificación respecto al pedido original.
 - Producto incompleto o con elementos faltantes de fábrica: Falta documentación técnica, piezas necesarias o embalaje original desde el origen.
- ❌ Motivos para RECHAZAR una solicitud de devolución:
 - Daños ocasionados durante el transporte: Si el transporte no estaba asegurado  
 - contratado directamente por la empresa, no se asume responsabilidad por los daños ocurridos durante el envío.
 - Manipulación indebida por parte del cliente: Instalación incorrecta, modificaciones o uso inapropiado del componente.
 - Superación del plazo máximo para devoluciones: La solicitud se presenta fuera del periodo establecido por la política de devoluciones (por ejemplo, 14 días naturales).
- 📧 Contestar el correo

Requisitos
- crear un flujo con al menos 3 pasos utilizando cualquiera de las herramientas usadas en clase .
- un pequeño documento justificando los prompts y el desarrollo realizado.


## 1. Instalación e imports

In [1]:
!pip install langchain langchain-openai dotenv

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import SystemMessage, HumanMessage
from dotenv import load_dotenv
import os

load_dotenv()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.56
    Uninstalling langchain-core-0.3.56:
      Successfully uninstalled langchain-core-0.3.56


False

## 2. Definición de prompts

In [59]:
system_prompt_reader_template = """
Eres un asistente de atención al cliente para Componentes Intergalácticos Industriales S.A. (al estilo star wars)

Tu tarea es analizar emails de solicitud de devolución

Extraelos datos relevantes:
- número de pedido (Unknow en caso de no haber sido informado)
- nombre del cliente o empresa (Unknow en caso de no haber sido informado)
- email
- motivo de la solicitud
- productos solicitados (array de productos)
- si se ha superado un plazo de 14 día naturales para devoluciones
- si hay defecto de fabricación en el o los productos reclamados (flase en caso de no haber sido informado)
- si se entregaron componentes incorrecto(false en caso de no haber sido informado)
- si el producto vino incompleto desde fábrica (false en caso de no haber sido informado)
- si hubo daños ocasionados durante el transporte (false en caso de no haber sido informado)
- si el envío fue asegurado por la empresa (false en caso de no haber sido informado)
- si hubo manipulación indebida por el cliente (false en caso de no haber sido informado)

Solo regresa la información en formato JSON. Contesta solo con el json

aqui tienes el email: {mail}
"""
system_prompt_reader = PromptTemplate(
    input_variables=["mail"],
    template=system_prompt_reader_template,
)

system_prompt_acceptance_template = """
Eres un asistente de atención al cliente para Componentes Intergalácticos Industriales S.A. (al estilo star wars)
Tu tarea es analizar el objeto json de entrada y decidir si se acepta, se rechaza o se solicita más información de acurdo a las siguientes reglas:

✅ ACEPTAR si:
- Hay defecto de fabricación.
- Se entregaron componentes incorrectos.
- El producto vino incompleto desde fábrica.

❌ RECHAZAR si:
- Si Los daños ocurrieron durante el transporte y el envío no fue asegurado por la empresa.
- Hubo manipulación indebida por parte del cliente.
- Se ha superado el plazo de 14 días naturales para devoluciones.

Resoponde sobre el json de entrada agregando en campo "resultado" si se acepta, se rechaza o se solicita más información y el motivo en el campo "motivo", priorizar motivos de rechazo sobre los de aceptación. Contesta solo con el json

aqui tienes el json: {json_info}
"""
system_prompt_acceptance = PromptTemplate(
    input_variables=["json_info"],
    template=system_prompt_acceptance_template,
)

system_prompt_mail_template = """
Eres un asistente de atención al cliente para Componentes Intergalácticos Industriales S.A. (al estilo star wars)

Tu tarea es analizar el objeto json de entrada donde se informa en el campo "resultado" si una solicitud de devolución es aceptada, rechazada o se solicita más información
En el campo "motivo" para indicar el motivo del rechazo

Y redacta un correo de acuerdo a la información del json de entrada respetando el resultado y motivo por encima de todo, mandalo a nombre del equipo de incidencias, nuestro correo: incidencias@cii.com, holofono: 87456-5465413-3131

Mantén un tono formal y en personaje del mundo de star wars

Contesta solo con el correo a enviar.

aqui tienes el json: {decision_json}
"""

system_prompt_mail = PromptTemplate(
    input_variables=["decision_json"],
    template=system_prompt_mail_template,
)

## 3. Construcción del chain

In [54]:
chat = ChatOpenAI(model="gpt-4o")
# 3.1 Lectura del correo
email_chain = LLMChain(
    llm=chat,
    prompt=system_prompt_reader
    )
# 3.2 Decisión de aceptación
acceptance_chain = LLMChain(
    llm=chat,
    prompt=system_prompt_acceptance)

# 3.3 Redacción del correo
mail_chain = LLMChain(
    llm=chat,
    prompt=system_prompt_mail
)


## 4. Flujo

### 4.1 Ejemplo 1

In [60]:
email_cliente = """
Asunto: Solicitud de reemplazo por daños en transporte – Pedido #D347-STELLA
Estimado equipo de Componentes Intergalácticos Industriales S.A.,
Me pongo en contacto con ustedes como cliente reciente para comunicar una incidencia relacionada con el pedido #D347-STELLA, correspondiente a un lote de condensadores de fluzo modelo FX-88, destinados a un proyecto estratégico de gran envergadura: la construcción de la Estrella de la Muerte.
Lamentablemente, al recibir el envío, observamos que varios de los condensadores presentaban daños visibles y no funcionales. Tras revisar el estado del embalaje y consultar con el piloto de carga, todo indica que la mercancía sufrió una caída durante el transporte interestelar.
Dado que estos componentes son críticos para la activación del núcleo central del sistema de rayos destructores, les solicitamos con carácter urgente el reemplazo inmediato de las unidades defectuosas, así como una revisión de los protocolos de embalaje y transporte para evitar que algo así vuelva a ocurrir.
Adjunto imágenes del estado de los condensadores y el albarán de entrega sellado por nuestro droide de recepción.
Agradezco de antemano su pronta atención a este asunto. Quedamos a la espera de su respuesta para coordinar el reemplazo.
Atentamente,
Darth Márquez
Departamento de Ingeniería Imperial
Sector de Proyectos Especiales
Contacto: dmarquez@imperiumgalactic.net
Holofono: +34 9X9 123 456
"""

In [56]:
# Paso 1
parsed_info = email_chain.invoke(input= email_cliente)
print(parsed_info)



{'mail': '\nAsunto: Solicitud de reemplazo por daños en transporte – Pedido #D347-STELLA\nEstimado equipo de Componentes Intergalácticos Industriales S.A.,\nMe pongo en contacto con ustedes como cliente reciente para comunicar una incidencia relacionada con el pedido #D347-STELLA, correspondiente a un lote de condensadores de fluzo modelo FX-88, destinados a un proyecto estratégico de gran envergadura: la construcción de la Estrella de la Muerte.\nLamentablemente, al recibir el envío, observamos que varios de los condensadores presentaban daños visibles y no funcionales. Tras revisar el estado del embalaje y consultar con el piloto de carga, todo indica que la mercancía sufrió una caída durante el transporte interestelar.\nDado que estos componentes son críticos para la activación del núcleo central del sistema de rayos destructores, les solicitamos con carácter urgente el reemplazo inmediato de las unidades defectuosas, así como una revisión de los protocolos de embalaje y transporte 

In [57]:
# Paso 2
decision_result = acceptance_chain.invoke(input= parsed_info["text"])
print(decision_result)


{'json_info': '```json\n{\n    "número de pedido": "D347-STELLA",\n    "nombre del cliente o empresa": "Darth Márquez",\n    "email": "dmarquez@imperiumgalactic.net",\n    "motivo de la solicitud": "Reemplazo por daños en transporte",\n    "productos solicitados": ["condensadores de fluzo modelo FX-88"],\n    "superó plazo de 14 días para devoluciones": false,\n    "defecto de fabricación": false,\n    "entrega de componentes incorrectos": false,\n    "producto incompleto desde fábrica": false,\n    "daños en transporte": true,\n    "envío asegurado": false,\n    "manipulación indebida por el cliente": false\n}\n```', 'text': '```json\n{\n    "número de pedido": "D347-STELLA",\n    "nombre del cliente o empresa": "Darth Márquez",\n    "email": "dmarquez@imperiumgalactic.net",\n    "motivo de la solicitud": "Reemplazo por daños en transporte",\n    "productos solicitados": ["condensadores de fluzo modelo FX-88"],\n    "superó plazo de 14 días para devoluciones": false,\n    "defecto de 

In [61]:
# Paso 3
respuesta_final = mail_chain.invoke(input= decision_result["json_info"])
print(respuesta_final['text'])

Asunto: Solicitud de Devolución - Pedido D347-STELLA - Darth Márquez

Estimada Darth Márquez,

Reciban un saludo cordial de parte del equipo de incidencias de Componentes Intergalácticos Industriales S.A. Esperamos que este mensaje le encuentre en buen estado.

Nos referimos a su reciente solicitud de devolución correspondiente al pedido D347-STELLA, relacionado con los condensadores de fluzo modelo FX-88 solicitados. Hemos llevado a cabo un análisis exhaustivo de su caso y, lamentablemente, debemos informar que su solicitud ha sido rechazada.

Motivo del rechazo: La solicitud de devolución ha sido denegada debido a que el envío no ha sido asegurado. Los daños reportados, aunque lamentables, no pueden ser cubiertos bajo nuestra política de devoluciones sin una póliza de seguro que ampare los riesgos de transporte. 

Entendemos la preocupación que esta situación pueda causar y nos comprometemos a seguir trabajando para mejorar constantemente nuestros servicios comunitarios por la galaxi

### 4.2 Ejemplo 2

In [62]:
email_cliente = """
Asunto: Solicitud de reemplazo – Pedido #D347-STELLA
Componentes Intergalácticos Industriales S.A.,
Relacionado con el pedido #D347-STELLA, he solicitado 4 transitores de cristales Kuarsom.
Sin embargo, el día de ayer hemos recibido solo 3 y de cristales Limnon.
Agradecería nos aceptaran la devolución y pocederemos a solicitar el material que necesitamos.
Atentamente,
Lukko Cielos
Contacto: lukkoc@resistenciaFTW.org
Holofono: +34 564-1656-165
"""

In [63]:
parsed_info = email_chain.invoke(input= email_cliente)
decision_result = acceptance_chain.invoke(input= parsed_info["text"])
respuesta_final = mail_chain.invoke(input= decision_result["json_info"])
print(respuesta_final['text'])

Asunto: Actualización sobre tu solicitud de devolución - Pedido D347-STELLA

Estimado Lukko Cielos,

Desde los confines de nuestra base en Componentes Intergalácticos Industriales S.A., te saludamos en esta galaxia lejana. Queremos agradecerte tu paciencia y confianza en nosotros, así como informar sobre el estado de tu solicitud de devolución para el pedido número D347-STELLA.

Hemos revisado detalladamente tu caso donde informas que has recibido un número incorrecto de transitores, y además, los transitores enviados eran incorrectos. Tras nuestro análisis, hemos determinado que debido a que el producto recibió componentes incorrectos y está incompleto, procederemos a aceptar la devolución del mismo.

Apreciamos que te tomes el tiempo de cuidar estos detalles, ya que nos ayuda a mejorar la calidad de nuestros servicios. Te proporcionaremos más instrucciones sobre cómo proceder con el envío de los componentes para que podamos resolver esta situación a la mayor brevedad posible.

Si tie

## 5. Concluciones y justificaciones

#### Justificación del flujo
Tras intentar algunos de los flujos que vimos en clase, obtuve varios errores y problemas para ser ejecutados en el notebook, por ese motivo opté por un flujo simple como el primero que vimos en clase.
Separé el problema en:

1. Extracción de información
2. Evaluación de la solicitud
3. Generación del correo

Justifico esta separación y la presentación de los resultados en JSON en la comunicación del punto 1 al 2 y del 2 al 3, porque estoy pensando que se ha modularizado. Mientras que el primer punto es bueno para obtener la información del correo, el segundo podría pasar por un sistema previo. Por ejemplo, si está informado el número de pedido, antes podría ir a la base de datos del CII y completar la información más fidedigna en vez de fiarnos de lo que dice el correo: "Ayer recibí...", cuando en nuestra base de datos consta que llegó hace 10 días, por ejemplo. De este modo se modulariza y es más flexible.

En cuanto a los prompts, aunque no lo parezca en el resultado final, tuve que hacer varias iteraciones (perdón por el consumo de la API key), pues es un poco puñetera. En más de un prompt tuve que reiterar para que hiciera caso de contestar solo con JSON y que en el correo respetara la decisión de aceptación o rechazo del punto anterior.

La estrategia de prompt que mejor me sirvió fue la de RolePlay, pues además de leer y entender bien los mensajes, también supo redactar en el mismo tono del universo de ***Star Wars***.

#### Conclusiones

Ha sido muy interesante el ejercicio y, si hubiera tenido más tiempo, me hubiera gustado mejorar el sistema, como la validación de fechas, ver si acepta una conversación del correo (petición inicial más respuestas entre la IA y el cliente), usar tools, por ejemplo simulando la llamada a la base de datos que comento en la justificación. Y más cosas que sería muy interesante analizar.

También puedo ver lo que comentaba en clase, cómo se puede llegar a romper fácilmente si los prompts cambian en la IA y cómo puede haber variables externas que lo modifiquen.